In [1]:
import numpy as np
import cv2

import os
import glob
from scipy.ndimage import rotate
from multiprocessing import Pool
from functools import partial

In [10]:
size = 860
shift = 12

In [3]:
def mapping(x, y, theta):
    pfish = np.zeros((2, 1))
    pfish[0] = y + shift * np.sin(theta)
    pfish[1] = x - shift * np.cos(theta)
    return np.around(pfish).astype(int)

In [4]:
def filter(ind, theta):
    i = ind // size
    j = ind % size
    fishcoord = mapping(j, i, theta)
    fishcoord[fishcoord >= size] = 0
    fishcoord[fishcoord < 0] = 0
    return fishcoord

In [7]:
translate_i = np.zeros((860, 860), int)
translate_j = np.zeros((860, 860), int)
pool = Pool()
coords = pool.map(filter, range(860*860))
for ind in range(860*860):
    i = ind // 860
    j = ind % 860
    translate_i[i, j] = coords[ind][0]
    translate_j[i, j] = coords[ind][1]

In [11]:
pool = Pool()
for x in range(3):
    theta = x * -2 * np.pi / 3 + np.pi / 2
    translate_i = np.zeros((860, 860), int)
    translate_j = np.zeros((860, 860), int)
    coords = pool.map(partial(filter, theta=theta), range(860*860))
    for ind in range(860*860):
        i = ind // 860
        j = ind % 860
        translate_i[i, j] = coords[ind][0]
        translate_j[i, j] = coords[ind][1]
    cam = x * 2
    np.save(f'translate_i_{cam}', translate_i)
    np.save(f'translate_j_{cam}', translate_j)


In [8]:
img = cv2.imread('/home/shane/0.jpg')
transform_i = np.load('/home/shane/stitching/image_processing/transform_i_0.npy')
transform_j = np.load('/home/shane/stitching/image_processing/transform_j_0.npy')
transform_img = img[transform_i, transform_j]
translate_img = transform_img[translate_i, translate_j]
cv2.imshow('test', translate_img)
cv2.waitKey(0)
cv2.destroyAllWindows()